# **TUGAS BESAR BAGIAN B ARTIFICIAL NEURAL NETWORK**


13521220 - Febryan Arota Hia

13521132 - Dhanika Novlisariyanti

13521153 - Made Debby Almadea Putri

13521155 - Kandida Edgina Gunawan

## **Import Library**

In [ ]:
import numpy as np
import json
import pickle
import random
import csv
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
class Layer:
  def __init__ (self, neuron: int, weight : np.array, bias: float, activation_func: str):
      self.neuron = neuron
      self.weight= weight
      self.deltaWeight = np.array(np.zeros_like(weight))
      self.bias = bias
      self.inputLayer = None
      self.outputLayer = None
      self.activation_function = activation_func
      valid_function = ["linear", "relu", "sigmoid", "softmax"]
      if activation_func not in valid_function:
        raise Exception("Invalid function!")

  def o_i(self, input: np.array):
    self.inputLayer = input
    net = np.dot(input, self.weight[1:]) + self.bias
    match self.activation_function:
      case "relu":
        self.outputLayer = np.maximum(0, net)
      case "linear":
        self.outputLayer = net
      case "sigmoid":
        self.outputLayer = (1/(1+np.exp((-1)*net)))
      case "softmax":
        self.outputLayer =  np.exp(net - np.max(net)) / np.sum(np.exp(net - np.max(net)))
    return self.outputLayer

  def derivative(self, output, target):
    match self.activation_function:
      case "relu":
        return (output > 0).astype(float)
      case "linear":
        return 1
      case "sigmoid":
        return (output * (1 - output))
      case "softmax":
        return output - target

  def getOutputGradient (self, target: np.array):
    bias_term = np.ones((self.inputLayer.shape[0], 1))
    inputs_with_bias = np.hstack([bias_term, np.array(self.inputLayer)])

    derivative = self.derivative(self.outputLayer, target)
    if self.activation_function != "softmax":
      err = np.subtract(self.outputLayer, target) * derivative
    else:
      err = derivative

    gradient = np.dot(inputs_with_bias.T, err)
    return gradient, err

  def getHiddenGradient(self, err_term: np.array):
    bias_term = np.ones((self.inputLayer.shape[0], 1))
    inputs_with_bias = np.hstack([bias_term, np.array(self.inputLayer)])

    derivative = self.derivative(self.outputLayer, err_term)
    if self.activation_function != "softmax":
      err = err_term * derivative
    else:
      err = derivative

    gradient = np.dot(inputs_with_bias.T, err)
    return gradient, err

  def updateDeltaWeight(self, learning_rate: float, gradien: np.array):
      # print("==============UPDATE DELTA WEIGHT====================")
      # print(f"Delta weight sekarang {self.deltaWeight}")
      # print(f"Gradien: {gradien}")

      self.deltaWeight -= (learning_rate * gradien)

      # print(f"Delta weight setelah update sekarang {self.deltaWeight}")

  def updateWeight(self):
      # print("==============UPDATE WEIGHT======================")
      # print(f"Weight lama: {self.weight}")
      # print(f"Delta weight: {self.deltaWeight}")

      self.weight += self.deltaWeight
      self.bias = self.weight[0]

      # print(f"Update weight baru jadi: {self.weight}")

      self.deltaWeight = np.array(np.zeros_like(self.weight))

In [ ]:
class BackwardPropagation:
    def __init__(self, _layers: list, learning_rate: float, batch_size: int, max_iteration: int, error_threshold: float):
      self.layers = _layers
      self.output = None
      self.num_of_layers = len(_layers)
      self.num_of_output_neuron = None
      self.final_weights = None

      self.learning_rate = learning_rate
      self.batch_size = batch_size
      self.max_iteration = max_iteration
      self.error_threshold = error_threshold
      self.stopped_by = None

    def addLayer(self, layer: Layer):
      self.layers.append(layer)
      self.num_of_layers += 1
      self.num_of_output_neuron = layer.neuron


    def forward(self, _input1 : np.array):
      # print("\nInput =================\n")
      self.output = _input1
      for l in self.layers:
        # print(f"Input : {self.output}")
        self.output = np.array(l.o_i(self.output))
        # print(f"Output: {self.output}")


    def backward(self, _target1):
      err_term = None
      reversed_layers = reversed(self.layers)
      for index, layer in enumerate(reversed_layers):
          if(index == 0):
            # print(f"Getting Output Layer Gradient for: {layer.activation_function}")

            gradient, err_term = layer.getOutputGradient(_target1)
          else:
            # print(f"Getting Hidden Layer Gradient for: {layer.activation_function}")

            err_term = np.dot(err_term, self.layers[len(self.layers) - index].weight.T)[:, 1:]
            gradient, err_term = layer.getHiddenGradient(err_term)

          layer.updateDeltaWeight(self.learning_rate, gradient)

    def fit(self, _inputs: np.array, _targets: np.array):
      iteration = 0
      curr_error = np.inf

      while(iteration < self.max_iteration and curr_error > self.error_threshold):
        print("\n\n", iteration / self.max_iteration * 100, "%")
        print("======================= ITERATION", iteration, "=======================")

        curr_error = 0
        for i in range(len(_inputs)):
          # print("=======================FORWARD PROPAGATION===========================")
          self.forward(np.array([_inputs[i]]))

          # print("=======================BACKWARD PROPAGATION===========================")
          self.backward(np.array([_targets[i]]))

          if(i == len(_inputs) - 1 or ((i + 1 - self.batch_size) % self.batch_size == 0)):
            print("UPDATE WEIGHT")
            for layer in self.layers:
              print()
              layer.updateWeight()
              print(layer.weight)

          if(self.layers[-1].activation_function == "softmax"):
            curr_error += 0
            for j in range(len(_targets[i])):
                if _targets[i][j] == 1:
                  # print(f"prev error : {curr_error}")

                  curr_error += (-np.log(self.output[0][j]))

                  # print(f"curr error: {curr_error} ")
                  break
          else:
            for j in range(len(_targets[i])):
                # print(f"prev error : {curr_error}")

                curr_error += ((((self.output[0][j] - _targets[0][j]))**2) /2)

                # print(f"curr error: {curr_error} ")

            curr_error /= len(_targets[i])

        curr_error /= len(_inputs)

        print(f"FINAL ERROR : {curr_error}")

        # for layer in self.layers:
        #   layer.deltaWeight = np.array(np.zeros_like(layer.weight))

        iteration += 1

      if(iteration == self.max_iteration):
        self.stopped_by = "max_iteration"
      else:
        self.stopped_by = "error_threshold"

      self.final_weights = []
      for layer in self.layers:
        self.final_weights.append(layer.weight)

    def predict(self, _inputs: np.array):
      self.forward(_inputs)
      print(f"Final weights: {self.final_weights}")
      print(f"Final output: {self.output}")
      max_values = self.output.max(axis = 1)
      mask = (self.output.T == max_values).T
      result = mask.astype(int)
      return result

    def export_model(self, name='bp-model.pkl'):
      with open(name, 'wb') as file:
        pickle.dump(self, file)

    def import_model(self, name):
      with open(name, 'rb') as file:
        loaded_model = pickle.load(file)

      return loaded_model

    def compare(self, expected_stopped_by, expected_final_weights, max_sse=(pow(10, -7))):
      sse = 0
      for i in range(len(self.final_weights)):
        for j in range(len(self.final_weights[i])):
          for k in range(len(self.final_weights[i][j])):
            sse += pow(self.final_weights[i][j][k] - expected_final_weights[i][j][k], 2)

      return self.stopped_by == expected_stopped_by, sse <= max_sse, sse

In [ ]:
class ModelConfig:
  def __init__(self, _input):
    self.layers = _input["case"]["model"]['layers']
    self.input = np.array(_input["case"]["input"])
    self.initial_weights = _input["case"]["initial_weights"]
    self.target = _input["case"]["target"]

    # learning parameters
    self.learning_rate = _input["case"]["learning_parameters"]["learning_rate"]
    self.batch_size = _input["case"]["learning_parameters"]["batch_size"]
    self.max_iteration = _input["case"]["learning_parameters"]["max_iteration"]
    self.error_threshold = _input["case"]["learning_parameters"]["error_threshold"]

    # expected
    self.expected_stopped_by = _input["expect"]["stopped_by"]
    self.expected_final_weights = _input["expect"]["final_weights"]


In [ ]:
class IO:
  def read_json(self, file: str):
    input = open(file, "r")
    input = json.load(input)

    config = ModelConfig(input)

    return config

  def read(self, file: str):
    config = self.read_json(file)

    model = BackwardPropagation([], config.learning_rate, config.batch_size, config.max_iteration, config.error_threshold)
    for i in range(len(config.layers)):
      neuron = config.layers[i]['number_of_neurons']
      activation_func = config.layers[i]['activation_function']
      weight = np.array(config.initial_weights[i])

      bias = np.array(config.initial_weights[i][0])
      layer = Layer(neuron, weight, bias, activation_func)
      model.addLayer(layer)

    return config, model

In [ ]:
def print_compare(config, model):
    print("\n=======================COMPARE=======================")
    print("STOPPED BY:", model.stopped_by)
    print("EXPECTED STOPPED BY:", config.expected_stopped_by)

    print("FINAL WEIGHTS:")
    print(model.final_weights)
    print("EXPECTED FINAL WEIGHTS:")
    print(config.expected_final_weights)

    same_stopped_by, same_final_weights, sse = model.compare(config.expected_stopped_by, config.expected_final_weights)

    print("SSE:", sse)

    print("PASSED:", same_stopped_by and same_final_weights)

## **TEST CASE**

In [ ]:
tc_folder = "/content/drive/MyDrive/IF3270 - Tugas Besar ML/Tubes B/tc/"

In [ ]:
io = IO()
config, model = io.read(tc_folder + "linear.json")
model.fit(config.input, config.target)
print_compare(config, model)



 0.0 %
======================= ITERATION 0 =======================
UPDATE WEIGHT

[[ 0.22  0.36  0.11]
 [ 0.64  0.3  -0.89]
 [ 0.28 -0.7   0.37]]
FINAL ERROR : 0.8022222222222221

=======================COMPARE=======================
STOPPED BY: max_iteration
EXPECTED STOPPED BY: max_iteration
FINAL WEIGHTS:
[array([[ 0.22,  0.36,  0.11],
       [ 0.64,  0.3 , -0.89],
       [ 0.28, -0.7 ,  0.37]])]
EXPECTED FINAL WEIGHTS:
[[[0.22, 0.36, 0.11], [0.64, 0.3, -0.89], [0.28, -0.7, 0.37]]]
SSE: 1.9451892438311082e-32
PASSED: True


In [ ]:
io = IO()
config, model = io.read(tc_folder + "linear_two_iteration.json")
model.fit(config.input, config.target)
print_compare(config, model)



 0.0 %
======================= ITERATION 0 =======================
UPDATE WEIGHT

[[ 0.22  0.36  0.11]
 [ 0.64  0.3  -0.89]
 [ 0.28 -0.7   0.37]]
FINAL ERROR : 0.8022222222222221


 50.0 %
======================= ITERATION 1 =======================
UPDATE WEIGHT

[[ 0.166  0.338  0.153]
 [ 0.502  0.226 -0.789]
 [ 0.214 -0.718  0.427]]
FINAL ERROR : 0.41558055555555556

=======================COMPARE=======================
STOPPED BY: max_iteration
EXPECTED STOPPED BY: max_iteration
FINAL WEIGHTS:
[array([[ 0.166,  0.338,  0.153],
       [ 0.502,  0.226, -0.789],
       [ 0.214, -0.718,  0.427]])]
EXPECTED FINAL WEIGHTS:
[[[0.166, 0.338, 0.153], [0.502, 0.226, -0.789], [0.214, -0.718, 0.427]]]
SSE: 1.6948183510607676e-32
PASSED: True


In [ ]:
io = IO()
config, model = io.read(tc_folder + "linear_small_lr.json")
model.fit(config.input, config.target)
print_compare(config, model)



 0.0 %
======================= ITERATION 0 =======================
UPDATE WEIGHT

[[ 0.1012  0.3006  0.1991]
 [ 0.4024  0.201  -0.7019]
 [ 0.1018 -0.799   0.4987]]
FINAL ERROR : 0.8022222222222221

=======================COMPARE=======================
STOPPED BY: max_iteration
EXPECTED STOPPED BY: max_iteration
FINAL WEIGHTS:
[array([[ 0.1012,  0.3006,  0.1991],
       [ 0.4024,  0.201 , -0.7019],
       [ 0.1018, -0.799 ,  0.4987]])]
EXPECTED FINAL WEIGHTS:
[[[0.1008, 0.3006, 0.1991], [0.402, 0.201, -0.7019], [0.101, -0.799, 0.4987]]]
SSE: 9.600000000000107e-07
PASSED: False


In [ ]:
io = IO()
config, model = io.read(tc_folder + "mlp.json")
model.fit(config.input, config.target)
print_compare(config, model)



 0.0 %
======================= ITERATION 0 =======================
UPDATE WEIGHT

[[ 0.08592   0.32276 ]
 [-0.33872   0.46172 ]
 [ 0.449984  0.440072]]

[[ 0.2748    0.188   ]
 [ 0.435904 -0.53168 ]
 [ 0.68504   0.7824  ]]
FINAL ERROR : 0.160644

=======================COMPARE=======================
STOPPED BY: max_iteration
EXPECTED STOPPED BY: max_iteration
FINAL WEIGHTS:
[array([[ 0.08592 ,  0.32276 ],
       [-0.33872 ,  0.46172 ],
       [ 0.449984,  0.440072]]), array([[ 0.2748  ,  0.188   ],
       [ 0.435904, -0.53168 ],
       [ 0.68504 ,  0.7824  ]])]
EXPECTED FINAL WEIGHTS:
[[[0.08592, 0.32276], [-0.33872, 0.46172], [0.449984, 0.440072]], [[0.2748, 0.188], [0.435904, -0.53168], [0.68504, 0.7824]]]
SSE: 2.1570415377137042e-32
PASSED: True


In [ ]:
io = IO()
config, model = io.read(tc_folder + "relu_b.json")
model.fit(config.input, config.target)
print_compare(config, model)



 0.0 %
======================= ITERATION 0 =======================
UPDATE WEIGHT

[[-0.211   0.105   0.885 ]
 [ 0.3033  0.5285  0.3005]
 [-0.489  -0.905   0.291 ]]
FINAL ERROR : 0.25048055555555554

=======================COMPARE=======================
STOPPED BY: max_iteration
EXPECTED STOPPED BY: max_iteration
FINAL WEIGHTS:
[array([[-0.211 ,  0.105 ,  0.885 ],
       [ 0.3033,  0.5285,  0.3005],
       [-0.489 , -0.905 ,  0.291 ]])]
EXPECTED FINAL WEIGHTS:
[[[-0.211, 0.105, 0.885], [0.3033, 0.5285, 0.3005], [-0.489, -0.905, 0.291]]]
SSE: 3.851859888774472e-33
PASSED: True


In [ ]:
io = IO()
config, model = io.read(tc_folder + "sigmoid.json")
model.fit(config.input, config.target)
print_compare(config, model)



 0.0 %
======================= ITERATION 0 =======================
UPDATE WEIGHT

[[0.29297259 0.09580488]
 [0.19280794 0.60482098]
 [0.80367835 0.29308146]]
FINAL ERROR : 0.11223794618876645


 10.0 %
======================= ITERATION 1 =======================
UPDATE WEIGHT

[[0.28601527 0.09165692]
 [0.18563174 0.60964881]
 [0.80737589 0.28617965]]
FINAL ERROR : 0.11208327034025578


 20.0 %
======================= ITERATION 2 =======================
UPDATE WEIGHT

[[0.27912837 0.08755599]
 [0.17847183 0.61448329]
 [0.81109236 0.2792947 ]]
FINAL ERROR : 0.11193235769812637


 30.0 %
======================= ITERATION 3 =======================
UPDATE WEIGHT

[[0.27231218 0.08350195]
 [0.1713286  0.61932423]
 [0.81482749 0.27242675]]
FINAL ERROR : 0.1117852082751855


 40.0 %
======================= ITERATION 4 =======================
UPDATE WEIGHT

[[0.26556695 0.07949467]
 [0.16420247 0.62417143]
 [0.81858101 0.2655759 ]]
FINAL ERROR : 0.11164181977565399


 50.0 %
=================

In [ ]:
io = IO()
config, model = io.read(tc_folder + "softmax.json")
model.fit(config.input, config.target)
print_compare(config, model)



 0.0 %
======================= ITERATION 0 =======================
UPDATE WEIGHT

[[ 9.24700795e-02  9.07552947e-01 -1.00023026e-01]
 [-1.81928191e-01  7.81872927e-01  2.00055263e-01]
 [ 3.20933179e-01 -7.20997192e-01  3.00064013e-01]
 [ 4.04517952e-01  5.95468232e-01 -3.99986184e-01]
 [ 4.97213929e-01  5.02794590e-01  4.99991480e-01]
 [-6.18523605e-01  4.18580249e-01  5.99943355e-01]
 [-6.93072473e-01 -3.06948711e-01  7.00021184e-01]
 [ 7.79217419e-01  2.20846133e-01 -8.00063553e-01]
 [ 8.80271608e-01 -8.02112791e-02 -6.03290704e-05]]
UPDATE WEIGHT

[[ 0.1024255   0.90754514 -0.10997064]
 [-0.19974839  0.7818869   0.2178615 ]
 [ 0.33735962 -0.72101007  0.28365045]
 [ 0.39685228  0.59547424 -0.39232652]
 [ 0.48695985  0.50280263  0.51023753]
 [-0.61752806  0.41857947  0.59894859]
 [-0.67196698 -0.30696526  0.67893224]
 [ 0.75572263  0.22086455 -0.77658718]
 [ 0.89271588 -0.08022103 -0.01249485]]
UPDATE WEIGHT

[[ 0.09782506  0.90467606 -0.10250111]
 [-0.20733912  0.7771529   0.230186

In [ ]:
io = IO()
config, model = io.read(tc_folder + "softmax_two_layer.json")
model.fit(config.input, config.target)
print_compare(config, model)

Streaming output truncated to the last 5000 lines.

[[-0.26071695 -0.27652735 -0.6419525   0.35505213]
 [-0.54856538 -1.09603679 -1.26935962  0.61997664]
 [-0.38375567  1.35284003 -0.90817938 -1.15279356]]

[[-1.26481661  1.28481661]
 [-0.46525078  0.44525078]
 [ 1.11822836 -1.09822836]
 [-1.07964957  1.05964957]
 [ 0.96207063 -0.94207063]]
UPDATE WEIGHT

[[-0.26237578 -0.27248922 -0.64585005  0.35505213]
 [-0.5442856  -1.10645516 -1.25930392  0.61997664]
 [-0.38612779  1.35861456 -0.91375289 -1.15279356]]

[[-1.26299472  1.28299472]
 [-0.46414706  0.44414706]
 [ 1.12640099 -1.10640099]
 [-1.07721864  1.05721864]
 [ 0.96207063 -0.94207063]]
UPDATE WEIGHT

[[-0.26193028 -0.27358437 -0.64480315  0.35505213]
 [-0.54581368 -1.1026988  -1.2628948   0.61997664]
 [-0.38623917  1.35888835 -0.91401461 -1.15279356]]

[[-1.26348521  1.28348521]
 [-0.4649814   0.4449814 ]
 [ 1.12483978 -1.10483978]
 [-1.0791325   1.0591325 ]
 [ 0.96207063 -0.94207063]]
UPDATE WEIGHT

[[-0.26193028 -0.27358437 -0.6

## **IRIS DATASET PROCESSING**

In [ ]:
class CSVProcessing:
  def __init__(self, fileName: str, removeHeader: bool):
    self.fileName = fileName #filename.csv
    self.removeHeader = removeHeader
    self.inputs = None
    self.outputs = None
    self.targetLabels = None
    self.inputValidation = None
    self.outputValidation = None

  def readFile(self):
    file = open(self.fileName)
    csvreader = csv.reader(file)
    rows = []
    rows2 = []
    for index, row in enumerate(csvreader):
      if(index == 0 and self.removeHeader):
        continue
      if((index >= 41 and index <= 50) or (index >= 91 and index <= 100) or (index >= 141 and index <= 150)):
        rows2.append(row)
      else:
        rows.append(row)

    self.inputs = [[float(val) for val in row[:-1]] for row in rows]
    self.inputs = [row[1:] for row in self.inputs]
    self.inputValidation = [[float(val) for val in row[:-1]] for row in rows2]
    self.inputValidation = [row[1:] for row in self.inputValidation]
    self.outputs = [row[-1] for row in rows]
    self.outputValidation = [row[-1] for row in rows2]

  def labelEncoding(self):
    self.targetLabels = pd.Series(self.outputs).drop_duplicates().to_list()
    out = []
    for i in range(len(self.outputs)):
      temp = []
      for j in range (len(self.targetLabels)):
        index = self.targetLabels.index(self.outputs[i])
        if(j == index):
          temp.append(1)
        else:
          temp.append(0)
      out.append(temp)
    self.outputs = out
    out = []
    for i in range(len(self.outputValidation)):
      temp = []
      for j in range (len(self.targetLabels)):
        index = self.targetLabels.index(self.outputValidation[i])
        if(j == index):
          temp.append(1)
        else:
          temp.append(0)
      out.append(temp)
    self.outputValidation = out



### **Eksekusi Model Backpropagation from Scratch**

In [ ]:
# Build JSON for for MLP

#TODO BUILD THIS MLP WITH THIS REQ:
# 1 input layer, 1 hidden layer (4 neuron) , 1 output layer
# max_iter = 10000
# batch_size = 64
# learning_rate = 0.001
# activation function for hidden layer : relu
# activation function for output layer : softmax
# error threshold = 0.0001
csvProcessing = CSVProcessing(tc_folder + "iris.csv", True)
csvProcessing.readFile()
csvProcessing.labelEncoding()
input_size = len(csvProcessing.inputs[0])
print(f"input size = {input_size}")
num_of_neurons = len(csvProcessing.targetLabels)
print(num_of_neurons)


initial_weights = []
hidden_weight = []
temp = []
for i in range(input_size + 1):
  for j in range(4):
    temp.append(random.uniform(0, 1))
  hidden_weight.append(temp)
  temp = []

initial_weights.append(hidden_weight)
hidden_weight = []
temp = []

for i in range(5):
  for j in range(num_of_neurons):
    temp.append(random.uniform(-1, 1))
  hidden_weight.append(temp)
  temp = []
initial_weights.append(hidden_weight)
print(initial_weights)


data = {
    "case":{
        "model":{
            "input_size": input_size,
            "layers":[
                {
                    "number_of_neurons": 4,
                    "activation_function" : "relu"
                },
                {
                    "number_of_neurons" : num_of_neurons,
                    "activation_function": "softmax"
                }
            ]
        },
        "input": csvProcessing.inputs,
        "initial_weights" : initial_weights,
        "target" :csvProcessing.outputs,
        "learning_parameters" : {
            "learning_rate" : 0.01,
            "batch_size" : 64,
            "max_iteration" : 1000,
            "error_threshold" : 0.01
        }
    },
    "expect": {
        "stopped_by" : "-",
        "final_weights" : [

        ]
    }
}
file = tc_folder + "test.json"
with open(file, "w") as json_file:
  json.dump(data, json_file)


input size = 4
3
[[[0.5470612599600158, 0.15226066588860898, 0.7082973380799785, 0.17498232693045013], [0.4875212614203578, 0.9108875131277396, 0.7201725399121135, 0.6328596792286846], [0.8293346909646029, 0.8410769480178495, 0.9411735589012454, 0.1454175233130336], [0.9160080512810566, 0.5035117426615268, 0.9931724644464084, 0.8111732900111492], [0.3710402394623745, 0.5221171143661977, 0.5854056928315182, 0.9736589201869078]], [[-0.954487600116783, 0.4126253874532204, -0.10758808602434478], [-0.3966679897012999, 0.8145189108374598, -0.18215510468281138], [-0.8161919408285372, -0.15001940825632598, 0.21003632786322024], [-0.021057107796111918, -0.8866329640855621, -0.7124003387907347], [0.7820437671658707, -0.20200261384942042, 0.5617224087316015]]]


In [ ]:
data = pd.read_csv(tc_folder + "iris.csv")
row_41_50 = data.iloc[40:50]
row_91_100 = data.iloc[90:100]
row_141_150 = data.iloc[140:150]
row_1_40 = data.iloc[0:10]
row_51_90 = data.iloc[50:90]
row_101_140 = data.iloc[100:140]
data_train = pd.concat([row_1_40, row_51_90, row_101_140])
data_test = pd.concat([row_41_50, row_91_100, row_141_150])

In [ ]:
y_train = data_train[['Species']]
X_train = data_train.drop(['Species', 'Id'], axis=1)
y_val = data_test[['Species']]
X_val = data_test.drop(['Species', 'Id'], axis=1)

scaler = StandardScaler()
scaler.fit(X_train)
scaled_data = scaler.transform(X_train)
X_train_scaled = pd.DataFrame(scaled_data, index=X_train.index, columns=X_train.columns)
scaled_data = scaler.transform(X_val)
X_val_scaled = pd.DataFrame(scaled_data, index=X_val.index, columns=X_val.columns)

encoder = LabelEncoder()
y_train_encoded = encoder.fit_transform(y_train.values.ravel())
y_val_encoded = encoder.transform(y_val.values.ravel())

print('y_train_encoded', y_train_encoded)
print('y_val_encoded', y_val_encoded)

num_classes = len(encoder.classes_)
y_train_categorical = to_categorical(y_train_encoded, num_classes)
y_val_categorical = to_categorical(y_val_encoded, num_classes)

y_train_encoded [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
y_val_encoded [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2]


In [ ]:
io = IO()
config, local_model = io.read(tc_folder + "test.json")

In [ ]:
local_model.fit(X_train_scaled.values, y_train_categorical)

Streaming output truncated to the last 5000 lines.
 [ 0.44606736  0.77763633  1.1555879   3.59508977]
 [ 0.31049327  0.94474217  0.81416092  2.94820398]]

[[ 3.35526733e+00 -1.22423488e+00 -2.78048275e+00]
 [-2.78486682e+00  2.55910354e+00  4.61459092e-01]
 [-1.20653383e+00 -1.62655322e-01  6.13014134e-01]
 [-4.99083315e-01 -9.41260472e-01 -1.79746624e-01]
 [-3.11365537e-03 -1.37089196e+00  2.51576918e+00]]
FINAL ERROR : 0.09073726618396609


 83.89999999999999 %
======================= ITERATION 839 =======================
UPDATE WEIGHT

[[ 2.43819587  0.01899626 -0.20891958 -0.18330814]
 [ 0.30707283  1.05388688  0.83909615 -0.58542166]
 [-0.41215181  0.35082755  0.20081295 -0.9492806 ]
 [ 0.45992603  0.77310925  1.15068331  3.57032801]
 [ 0.32168608  0.94071568  0.81022621  2.92836336]]

[[ 3.35606734e+00 -1.20207552e+00 -2.80344211e+00]
 [-2.78580395e+00  2.62094058e+00  4.00559194e-01]
 [-1.20657282e+00 -1.52471833e-01  6.02869632e-01]
 [-4.99094351e-01 -9.33822169e-01 -1.87173891

In [ ]:
print(local_model.final_weights)

[array([[ 2.34803198e+00,  3.31818845e-03, -2.22067821e-01,
        -1.06822373e-01],
       [ 3.09854006e-01,  1.10063142e+00,  8.83051397e-01,
        -6.12178746e-01],
       [-3.94741025e-01,  3.26006575e-01,  1.76901627e-01,
        -9.83636762e-01],
       [ 4.17980683e-01,  7.55054157e-01,  1.13559407e+00,
         3.68518636e+00],
       [ 3.14009906e-01,  9.54684022e-01,  8.25708011e-01,
         3.02920077e+00]]), array([[ 3.47351397, -1.22846376, -2.89450051],
       [-2.93050777,  2.61830779,  0.5478958 ],
       [-1.21281619, -0.1677414 ,  0.62438257],
       [-0.5009981 , -0.9350211 , -0.18407121],
       [-0.01028354, -1.31285612,  2.46490323]])]


In [ ]:
y_pred = local_model.predict(X_val_scaled.values)
print(f"y_pred: {y_pred}")
accuracy = accuracy_score(y_val_categorical, y_pred)
print(f"accuracy: {accuracy * 100} %")

Final weights: [array([[ 2.34803198e+00,  3.31818845e-03, -2.22067821e-01,
        -1.06822373e-01],
       [ 3.09854006e-01,  1.10063142e+00,  8.83051397e-01,
        -6.12178746e-01],
       [-3.94741025e-01,  3.26006575e-01,  1.76901627e-01,
        -9.83636762e-01],
       [ 4.17980683e-01,  7.55054157e-01,  1.13559407e+00,
         3.68518636e+00],
       [ 3.14009906e-01,  9.54684022e-01,  8.25708011e-01,
         3.02920077e+00]]), array([[ 3.47351397, -1.22846376, -2.89450051],
       [-2.93050777,  2.61830779,  0.5478958 ],
       [-1.21281619, -0.1677414 ,  0.62438257],
       [-0.5009981 , -0.9350211 , -0.18407121],
       [-0.01028354, -1.31285612,  2.46490323]])]
Final output: [[1.04967068e-06 9.52818289e-09 1.80077527e-09]
 [1.29022782e-07 6.20025027e-08 2.66482021e-09]
 [1.04967068e-06 9.52818289e-09 1.80077527e-09]
 [1.04967068e-06 9.52818289e-09 1.80077527e-09]
 [1.04967068e-06 9.52818289e-09 1.80077527e-09]
 [7.69447361e-07 1.25752172e-08 1.90842879e-09]
 [1.04967068e

### **Eksekusi SKLEARN MLP**

In [ ]:
print(f"Bias and weights layer 1: {initial_weights[0]}")
print(f"Bias and weights output layer: {initial_weights[1]}")

Bias and weights layer 1: [[0.5470612599600158, 0.15226066588860898, 0.7082973380799785, 0.17498232693045013], [0.4875212614203578, 0.9108875131277396, 0.7201725399121135, 0.6328596792286846], [0.8293346909646029, 0.8410769480178495, 0.9411735589012454, 0.1454175233130336], [0.9160080512810566, 0.5035117426615268, 0.9931724644464084, 0.8111732900111492], [0.3710402394623745, 0.5221171143661977, 0.5854056928315182, 0.9736589201869078]]
Bias and weights output layer: [[-0.954487600116783, 0.4126253874532204, -0.10758808602434478], [-0.3966679897012999, 0.8145189108374598, -0.18215510468281138], [-0.8161919408285372, -0.15001940825632598, 0.21003632786322024], [-0.021057107796111918, -0.8866329640855621, -0.7124003387907347], [0.7820437671658707, -0.20200261384942042, 0.5617224087316015]]


In [ ]:
model = Sequential()
model.add(Dense(4, input_dim=X_train.shape[1], activation='relu',
                kernel_initializer='random_uniform', bias_initializer='zeros'))
model.add(Dense(num_classes, activation='softmax',
                kernel_initializer='random_uniform', bias_initializer='zeros'))

# Compilation of the model
optimizer = SGD(learning_rate=0.01)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

#Setting weight (same with the one we made from scratch)
weights_layer1 = np.array(initial_weights[0][1:])  #weight at hidden layer
biases_layer1 = np.array(initial_weights[0][0])    #bias at hidden layer
weights_layer2 = np.array(initial_weights[1][1:])  #weight at output layer
biases_layer2 = np.array(initial_weights[1][0])       #bias at output layer

print("weights_layer1\n", weights_layer1)
print("biases_layer1\n", biases_layer1)
print("weights_layer2\n", weights_layer2)
print("biases_layer2\n", biases_layer2)
model.layers[0].set_weights([weights_layer1, biases_layer1])
model.layers[1].set_weights([weights_layer2, biases_layer2])

model.fit(X_train, y_train_categorical, batch_size=config.batch_size, epochs=1000, verbose=1)

loss, accuracy = model.evaluate(X_val, y_val_categorical, verbose=0)
print(f"Accuracy Keras Model: {accuracy * 100} %")

for layer in model.layers:
    weights, biases = layer.get_weights()
    print("Weights:", weights)
    print("Biases:", biases)


weights_layer1
 [[0.48752126 0.91088751 0.72017254 0.63285968]
 [0.82933469 0.84107695 0.94117356 0.14541752]
 [0.91600805 0.50351174 0.99317246 0.81117329]
 [0.37104024 0.52211711 0.58540569 0.97365892]]
biases_layer1
 [0.54706126 0.15226067 0.70829734 0.17498233]
weights_layer2
 [[-0.39666799  0.81451891 -0.1821551 ]
 [-0.81619194 -0.15001941  0.21003633]
 [-0.02105711 -0.88663296 -0.71240034]
 [ 0.78204377 -0.20200261  0.56172241]]
biases_layer2
 [-0.9544876   0.41262539 -0.10758809]
Epoch 1/1000
2/2 [==============================] - 0s 14ms/step - loss: 1.4741 - accuracy: 0.4778
Epoch 2/1000
2/2 [==============================] - 0s 9ms/step - loss: 1.3665 - accuracy: 0.4222
Epoch 3/1000
2/2 [==============================] - 0s 13ms/step - loss: 1.4740 - accuracy: 0.5444
Epoch 4/1000
2/2 [==============================] - 0s 9ms/step - loss: 1.1338 - accuracy: 0.4333
Epoch 5/1000
2/2 [==============================] - 0s 8ms/step - loss: 1.3849 - accuracy: 0.4222
Epoch 6/1000
2/2

In [ ]:
model.fit(X_train_scaled, y_train_categorical, batch_size=config.batch_size, epochs=1000, verbose=1)

loss, accuracy = model.evaluate(X_val_scaled, y_val_categorical, verbose=0)
print(f"Accuracy Keras Model: {accuracy * 100} %")

for layer in model.layers:
    weights, biases = layer.get_weights()
    print("Weights:", weights)
    print("Biases:", biases)

Epoch 1/1000
2/2 [==============================] - 0s 32ms/step - loss: 0.6841 - accuracy: 0.7222
Epoch 2/1000
2/2 [==============================] - 0s 21ms/step - loss: 0.6603 - accuracy: 0.7556
Epoch 3/1000
2/2 [==============================] - 0s 19ms/step - loss: 0.6370 - accuracy: 0.7667
Epoch 4/1000
2/2 [==============================] - 0s 10ms/step - loss: 0.6139 - accuracy: 0.7778
Epoch 5/1000
2/2 [==============================] - 0s 12ms/step - loss: 0.5929 - accuracy: 0.7778
Epoch 6/1000
2/2 [==============================] - 0s 12ms/step - loss: 0.5739 - accuracy: 0.7778
Epoch 7/1000
2/2 [==============================] - 0s 22ms/step - loss: 0.5581 - accuracy: 0.7889
Epoch 8/1000
2/2 [==============================] - 0s 19ms/step - loss: 0.5438 - accuracy: 0.7889
Epoch 9/1000
2/2 [==============================] - 0s 15ms/step - loss: 0.5308 - accuracy: 0.7889
Epoch 10/1000
2/2 [==============================] - 0s 14ms/step - loss: 0.5182 - accuracy: 0.8000
Epoch 11/